In [16]:
from arcgis.gis import GIS, Item
import os
from dotenv import load_dotenv, find_dotenv

import sys
sys.path.append('../azure_function_project')
from src import ComputerVision

# load the .env settings
load_dotenv('../azure_function_project/detect-weapon/.env')

True

In [12]:
gis_url = os.getenv('GIS_URL')
gis_username = os.getenv('GIS_USERNAME')
gis_password = os.getenv('GIS_PASSWORD')

feature_service_item_id = os.getenv('SURVEY_LAYER_ITEM_ID')

azure_key = os.getenv('AZURE_KEY')
azure_region_code = os.getenv('AZURE_REGION_CODE')

weapon_tags = ['weapon', 'gun', 'knife', 'axe']

feature_oid = 2  # need to get this from JSON payload

feature_service_item_id

'9a8ebbff0726445ebe0e3766b19c7f3c'

In [20]:
# instantiate an gis object instance using credentials
gis = GIS(gis_url, username=gis_username, password=gis_password)

# instantiate a layer object referencing the feature service layer
lyr_pt = Item(gis, feature_service_item_id).layers[0]

# get the attachment list for the feature 
attachment_lst = lyr_pt.attachments.get_list(feature_oid)

# from the first attachment, the one submitted from the survey, get the id to use
attachment_id = attachment_lst[0]['id']

# using the feature layer url combined with the feature oid and attachemnt oid, create the url for the attachement image
image_url = f'{lyr_pt.url}/{feature_oid}/attachments/{attachment_id}?token={gis._con.token}'

# determine if a weapon exists
vision = ComputerVision(azure_key, azure_region_code)
weapon_status = vision.image_tag_match(image_url, weapon_tags)

# get the feature object to work with
feat = lyr_pt.query(object_ids=f'{feature_oid}').features[0]

# update the weapon attribute in the feature with the returned status
feat.attributes['weapon'] = weapon_status

# push the updated feature to the feature service
update_resp = lyr_pt.edit_features(updates=[feat])

update_resp

{'addResults': [],
 'updateResults': [{'objectId': 2,
   'uniqueId': 2,
   'globalId': '7b3b0e08-7c90-461b-8673-b44579fe14af',
   'success': True}],
 'deleteResults': []}

In [9]:
f'{lyr_pt.url}/{feature_oid}/attachments/{attachment_id}?token={gis._con.token}'

'https://services8.arcgis.com/r5nMFFrM0YYutfJu/arcgis/rest/services/Azure_Weapon_Detection_azure/FeatureServer/0/2/attachments/2?token=rYgBgXS2osRcTxzmCY5QKaneN6YiZVjCuzORr8FGH-4xAHeEysErHVqeyuE3JAi1y5mUJ5BPOsSgIRNAnwzgQgiSxuYt_s0ebDB3PKjNRs0HO887Mlk6cODhcsHkwu9K'